In [ ]:
from jp_doodle import doodle_files
from jp_doodle import dual_canvas
import jp_proxy_widget
from jp_doodle import quantity_forest

In [ ]:
c = dual_canvas.DualCanvasWidget(width=800)
F = quantity_forest.FileSystemExplorer(c, "~/repos", degrees=45, background="#ddf")

In [ ]:
c

In [1]:
from jp_doodle import doodle_files
from jp_doodle import dual_canvas
import jp_proxy_widget
from chart_ipynb import pie
from jp_doodle import quantity_forest
from jp_doodle.quantity_forest import directory_usage
import os

class FileSystemChart(quantity_forest.FileSystemExplorer):

    def __init__(self, path, width=800, epsilon=0.02):
        path = os.path.expanduser(path)
        path = os.path.abspath(path)
        self.path = path
        self.current_path = path
        self.previous_path = path
        self.parent_path = {path: None}
        self.color_cache = {}
        self.usage_cache = {}
        self.id_to_data = {}
        self.expanded = {}
        self.epsilon = epsilon
        self.width = width
        
        members = self.directory_members(path)

        labels = [d['label'] for d in members]
        color = [d['color'] for d in members]
        value = [d['size'] for d in members]
        quantity_pie = pie.Pie()
        quantity_pie.add_dataset(labels, value, 'root', color = color)
        self.widget = quantity_pie
    
    def path_update(self, directory_name):
        path = directory_name
        if '/' not in path:
            path = self.current_path + '/' + directory_name
        else:
            directory_name = directory_name.split('/')[-1]
        if directory_name in self.usage_cache[self.current_path]:
            self.parent_path[path] = self.current_path
        self.previous_path = self.current_path
        self.current_path = path
        self.chart_update(path)
    
    def go_root(self):
        self.chart_update(self.path)
        self.previous_path = self.current_path
        self.current_path = self.path
    
    def go_previous(self):
        temp_path = self.previous_path
        self.previous_path = self.current_path
        self.current_path = temp_path
        self.chart_update(temp_path)
    
    def go_parent(self):
        temp_path = self.parent_path[self.current_path]
        if temp_path is None:
            return
        self.previous_path = self.current_path
        self.current_path = temp_path
        self.chart_update(temp_path)
    
    def chart_update(self, path):
        members = self.directory_members(path)
        labels = [d['label'] for d in members]
        color = [d['color'] for d in members]
        value = [d['size'] for d in members]
        newdataset = {'label':labels,
             'data':value,
             'backgroundColor':color}
        self.widget.js_init("""
    console.log(element.chart_info.chart.config);
    element.chart_info.chart.config.data.datasets.splice(0, 1);
    element.chart_info.chart.config.data.datasets.push(newDataset);
    element.chart_info.chart.config.data.labels = newDataset.label;
    element.chart_info.chart = new Chart(element.chart_info.context, element.chart_info.chart.config);
""", newDataset=newdataset)
        
    
    def setup_chart(self):
        self.widget.setup(width=self.width)
        return self.widget

In [2]:
t = FileSystemChart("~/Chart_ipynb")
t.setup_chart()

Pie(status='deferring flush until render')

In [3]:
t.path_update('chart_ipynb')

In [4]:
t.go_parent()